In [2]:
import pandas as pd
import numpy as np
from math import sqrt
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from pymlx import *

from keras.models import Sequential, Model
from keras.layers import Activation, Input, Dense, Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Flatten, GRU, LSTM
from keras.layers.advanced_activations import ELU
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import *
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

K.set_image_dim_ordering('th')

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def compute_mae(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    result = mean_absolute_error(y_pred=forecasts, y_true=y_denom)
    return result


def compute_mape_centerScale(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],1)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    sub_result = np.abs((y_denom - forecasts) / y_denom )
    result = (np.mean(sub_result))
    return result


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))


def test_mae(model):
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)
    return(compute_mae(model, X_train, y_train), compute_mae(model, X_test, y_test))


def report(train, test):
    train_mean = np.asarray(train).mean()
    train_std = np.asarray(train).std()
    test_mean = np.asarray(test).mean()
    test_std = np.asarray(test).std()
    train = str('Training MAE %f ± %f'% (train_mean, train_std))
    test = str('Testing MAE %f ± %f'% (test_mean, test_std))
    return(train, test)

In [4]:
# ndre_data = pd.read_csv('../HydroSatML/data/final_join_subbed_missing_soilM.csv')
# ndre_data_sub_30 = pd.read_csv('../HydroSatML/data/final_join_subbed_bare_soil.csv')
# ndre_data_sub_40 = pd.read_csv('../HydroSatML/data/final_join_subbed_bare_soil_40.csv')
# ndre_data_lag_40 = pd.read_csv('../data/final_join_subbed_bare_soil_with_lag_40_2.csv')
ndre_data_lag_40_EXACT = pd.read_csv('../data/final_join_subbed_bare_soil_with_lag_40_EXACT_NDRE.csv')

In [5]:
ndre_data_lag_40.columns

Index(['field', 'average_adjacent', 'stdev_adjacent', 'sensor', 'date',
       'sensor_full_name', 'depth_1', 'depth_2', 'depth_3', 'year',
       'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s',
       'irradiance.w_per_m.2', 'sand_1', 'sand_2', 'sand_3', 'silt_1',
       'silt_2', 'silt_3', 'clay_1', 'clay_2', 'clay_3', 'l3_depth_1',
       'l3_depth_2', 'l3_depth_3', 'l5_depth_1', 'l5_depth_2', 'l5_depth_3',
       'l7_depth_1', 'l7_depth_2', 'l7_depth_3', 'avg_soilM', 'avg_soilM_l3',
       'avg_soilM_l5', 'avg_soilM_l7', 'avg_clay', 'avg_sand', 'avg_silt'],
      dtype='object')

In [5]:
# keep_cols = ['avg_soilM', 'average_adjacent', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 
#              'irradiance.w_per_m.2', 'sand_1', 'sand_2', 'sand_3', 'silt_1', 'silt_2', 'silt_3', 'clay_1', 
#              'clay_2', 'clay_3']
keep_cols = ['avg_soilM', 'ndre_val', 'precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 
             'irradiance.w_per_m.2', 'avg_clay', 'avg_sand', 'avg_silt']

dataset = ndre_data_lag_40_EXACT[keep_cols]

In [6]:
X = dataset.iloc[:,1:]
X = X.values

y = dataset.iloc[:,0]
y = y.values

# split data into train and test sets
seed = 7
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

#### normalize

mean_X = X_train.mean(axis=0)
std_X = X_train.std(axis=0)

X_train = (X_train - mean_X) / std_X
X_test = (X_test - mean_X) / std_X

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

mean_y = y_train.mean(axis=0)
std_y = y_train.std(axis=0)

y_train = (y_train - mean_y) / std_y
y_test = (y_test - mean_y) / std_y

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

dim = X_train.shape[1]

### Building the NN

In [7]:
np.random.seed(84)
tf.set_random_seed(84)

def create_NN_model(units1=16,
                    units2=16,
                    units3=16,
                    units4=16,
                    layers=1,
                    epochs=50,
                    optimizer='adam',
                    activation=ELU(),
                    beta_1=0.9,
                    beta_2=0.99,
                    epsilon=1e-8,
                    decay=0,
                    schedule_decay=0.004,
                    rho=0.8,
                    lr=0.001,
                    momentum=0.9,
                    nesterov=False):
    
    if layers == 1:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(1, activation=activation))
    elif layers == 2:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(units2, activation=activation))
        model.add(Dense(1, activation=activation))
    elif layers == 3:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(units2, activation=activation))
        model.add(Dense(units3, activation=activation))
        model.add(Dense(1, activation=activation))
    elif layers == 4:
        model = Sequential()
        model.add(Dense(units1, input_dim=dim, activation=activation))
        model.add(Dense(units2, activation=activation))
        model.add(Dense(units3, activation=activation))
        model.add(Dense(units4, activation=activation))
        model.add(Dense(1, activation=activation))
    else:
        pass
    
    if(optimizer == 'SGD'):
        sgd = SGD(lr=lr, momentum=momentum, decay=decay, nesterov=nesterov)
    elif(optimizer == 'RMSprop'):
        sgd = RMSprop(lr=lr, rho=rho)
    elif(optimizer == 'Nadam'):
        sgd = Nadam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, schedule_decay=schedule_decay)
    elif(optimizer == 'Adagrad'):
        sgd = Adagrad(lr=lr, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adadelta'):
        sgd = Adadelta(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adam'):
        sgd = Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    elif(optimizer == 'Adamax'):
        sgd = Adamax(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    else:
        pass
    
    model.compile(loss='mae', optimizer=sgd, metrics=['mse'])
    
    return model

### Setting paramters

In [14]:
model = KerasRegressor(build_fn=create_NN_model, shuffle=True, verbose=0)

units1 = [4,8,26,32,64]
units2 = [4,8,16,32,64]
units3 = [4,8,16,32,64]
units4 = [4,8,16,32,64]

layers = [1,2,3,4]
epochs = [100]
batch_size = [1,2,3,4]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['relu', ELU()]
decay = np.arange(0,.1,0.001).tolist()
beta_1 = np.random.uniform(low=0.90, high=1.0, size=20).tolist()
beta_2 = np.random.uniform(0.90, 1.0, 20).tolist()
rho = np.random.uniform(0.80, 0.95, 20).tolist()
epsilon = [1e-09, 1e-08,1e-07, 1e-06, 1e-05, 1e-04]
schedule_decay = np.arange(0.001,0.01,0.0005).tolist()
lr = [10**k for k in range(-5, -1)]

param_grid = dict(units1=units1,
                 units2=units2,
                 units3=units3,
                 units4=units4,
                 layers=layers,
                 epochs=epochs,
                 optimizer=optimizer,
                 activation=activation,
                 lr=lr,
                 schedule_decay=schedule_decay,
                 epsilon=epsilon,
                 rho=rho,
                 beta_1=beta_1,
                 beta_2=beta_2,
                 decay=decay)

In [16]:
start_timing()
sweeper = random_sweep(
    X_train, y_train, 
    model, param_grid,
    scoring=compute_mae, 
    n_iter=75, n_jobs=1, 
    refit=False, cv=3, verbose=75)
report_timing()

Fitting 3 folds for each of 75 candidates, totalling 225 fits
[CV] units4=4, units3=64, units2=4, units1=64, schedule_decay=0.003, rho=0.8311758068665892, optimizer=Adagrad, lr=0.001, layers=4, epsilon=1e-06, epochs=100, decay=0.036000000000000004, beta_2=0.9835223929985658, beta_1=0.9793775053748464, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8> 
[CV]  units4=4, units3=64, units2=4, units1=64, schedule_decay=0.003, rho=0.8311758068665892, optimizer=Adagrad, lr=0.001, layers=4, epsilon=1e-06, epochs=100, decay=0.036000000000000004, beta_2=0.9835223929985658, beta_1=0.9793775053748464, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8>, score=0.045558, total=   2.9s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s
[CV] units4=4, units3=64, units2=4, units1=64, schedule_decay=0.003, rho=0.8311758068665892, optimizer=Adagrad, lr=0.001, layers=4, epsilon=1e-06, epochs=100, decay=0.036000000000000004, beta_2=0.9

[CV]  units4=16, units3=64, units2=64, units1=32, schedule_decay=0.0045000000000000005, rho=0.8265195577225888, optimizer=Adam, lr=0.001, layers=3, epsilon=1e-06, epochs=100, decay=0.06, beta_2=0.936669600294371, beta_1=0.9423346068550754, activation=relu, score=0.046249, total=   3.4s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:   36.3s remaining:    0.0s
[CV] units4=16, units3=64, units2=64, units1=32, schedule_decay=0.0045000000000000005, rho=0.8265195577225888, optimizer=Adam, lr=0.001, layers=3, epsilon=1e-06, epochs=100, decay=0.06, beta_2=0.936669600294371, beta_1=0.9423346068550754, activation=relu 
[CV]  units4=16, units3=64, units2=64, units1=32, schedule_decay=0.0045000000000000005, rho=0.8265195577225888, optimizer=Adam, lr=0.001, layers=3, epsilon=1e-06, epochs=100, decay=0.06, beta_2=0.936669600294371, beta_1=0.9423346068550754, activation=relu, score=0.039372, total=   3.9s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:   40.2s remaining:    0.0s
[CV] units

[CV]  units4=8, units3=32, units2=32, units1=4, schedule_decay=0.005, rho=0.8171340077735547, optimizer=Adadelta, lr=0.001, layers=2, epsilon=1e-08, epochs=100, decay=0.012, beta_2=0.9819408717834255, beta_1=0.9870151785241231, activation=relu, score=0.038948, total=   3.7s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.4min remaining:    0.0s
[CV] units4=16, units3=64, units2=16, units1=26, schedule_decay=0.009500000000000001, rho=0.9021771605381883, optimizer=Nadam, lr=0.0001, layers=2, epsilon=1e-06, epochs=100, decay=0.057, beta_2=0.9355739559428804, beta_1=0.9564524845117608, activation=relu 
[CV]  units4=16, units3=64, units2=16, units1=26, schedule_decay=0.009500000000000001, rho=0.9021771605381883, optimizer=Nadam, lr=0.0001, layers=2, epsilon=1e-06, epochs=100, decay=0.057, beta_2=0.9355739559428804, beta_1=0.9564524845117608, activation=relu, score=0.046218, total=   4.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:  1.5min remaining:    0.0s
[CV] units4=16, 

[CV]  units4=8, units3=16, units2=4, units1=8, schedule_decay=0.0055000000000000005, rho=0.9164706059234075, optimizer=Adagrad, lr=1e-05, layers=3, epsilon=1e-09, epochs=100, decay=0.022, beta_2=0.926171906571784, beta_1=0.9423346068550754, activation=relu, score=0.048038, total=   4.1s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  2.3min remaining:    0.0s
[CV] units4=8, units3=16, units2=4, units1=8, schedule_decay=0.0055000000000000005, rho=0.9164706059234075, optimizer=Adagrad, lr=1e-05, layers=3, epsilon=1e-09, epochs=100, decay=0.022, beta_2=0.926171906571784, beta_1=0.9423346068550754, activation=relu 
[CV]  units4=8, units3=16, units2=4, units1=8, schedule_decay=0.0055000000000000005, rho=0.9164706059234075, optimizer=Adagrad, lr=1e-05, layers=3, epsilon=1e-09, epochs=100, decay=0.022, beta_2=0.926171906571784, beta_1=0.9423346068550754, activation=relu, score=0.039531, total=   4.3s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:  2.4min remaining:    0.0s
[CV] un

[CV]  units4=16, units3=16, units2=8, units1=4, schedule_decay=0.0025, rho=0.8543654890157573, optimizer=RMSprop, lr=0.01, layers=2, epsilon=1e-07, epochs=100, decay=0.058, beta_2=0.9265849596843658, beta_1=0.9288119327973722, activation=relu, score=0.038608, total=   4.6s
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  3.4min remaining:    0.0s
[CV] units4=16, units3=32, units2=4, units1=32, schedule_decay=0.0065, rho=0.9471585230297452, optimizer=SGD, lr=0.001, layers=1, epsilon=1e-08, epochs=100, decay=0.011, beta_2=0.9875983196541405, beta_1=0.9423346068550754, activation=relu 
[CV]  units4=16, units3=32, units2=4, units1=32, schedule_decay=0.0065, rho=0.9471585230297452, optimizer=SGD, lr=0.001, layers=1, epsilon=1e-08, epochs=100, decay=0.011, beta_2=0.9875983196541405, beta_1=0.9423346068550754, activation=relu, score=0.046304, total=   4.4s
[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed:  3.5min remaining:    0.0s
[CV] units4=16, units3=32, units2=4, units1=32, sched

...

[CV]  units4=64, units3=64, units2=16, units1=4, schedule_decay=0.007500000000000001, rho=0.9471585230297452, optimizer=RMSprop, lr=0.01, layers=4, epsilon=1e-05, epochs=100, decay=0.038, beta_2=0.9643806747145691, beta_1=0.9793775053748464, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8>, score=0.040421, total=   9.9s
[CV] units4=64, units3=64, units2=16, units1=4, schedule_decay=0.007500000000000001, rho=0.9471585230297452, optimizer=RMSprop, lr=0.01, layers=4, epsilon=1e-05, epochs=100, decay=0.038, beta_2=0.9643806747145691, beta_1=0.9793775053748464, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8> 
[CV]  units4=64, units3=64, units2=16, units1=4, schedule_decay=0.007500000000000001, rho=0.9471585230297452, optimizer=RMSprop, lr=0.01, layers=4, epsilon=1e-05, epochs=100, decay=0.038, beta_2=0.9643806747145691, beta_1=0.9793775053748464, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8>, score=0.037666, total= 

[CV]  units4=4, units3=16, units2=32, units1=64, schedule_decay=0.0035, rho=0.8254104998646793, optimizer=Nadam, lr=0.01, layers=2, epsilon=1e-05, epochs=100, decay=0.096, beta_2=0.9374027733163808, beta_1=0.9270092329603724, activation=relu, score=0.049669, total=  10.3s
[CV] units4=4, units3=16, units2=32, units1=64, schedule_decay=0.0035, rho=0.8254104998646793, optimizer=Nadam, lr=0.01, layers=2, epsilon=1e-05, epochs=100, decay=0.096, beta_2=0.9374027733163808, beta_1=0.9270092329603724, activation=relu 
[CV]  units4=4, units3=16, units2=32, units1=64, schedule_decay=0.0035, rho=0.8254104998646793, optimizer=Nadam, lr=0.01, layers=2, epsilon=1e-05, epochs=100, decay=0.096, beta_2=0.9374027733163808, beta_1=0.9270092329603724, activation=relu, score=0.039326, total=  10.3s
[CV] units4=4, units3=16, units2=32, units1=64, schedule_decay=0.0035, rho=0.8254104998646793, optimizer=Nadam, lr=0.01, layers=2, epsilon=1e-05, epochs=100, decay=0.096, beta_2=0.9374027733163808, beta_1=0.92700

[CV]  units4=16, units3=8, units2=4, units1=32, schedule_decay=0.0055000000000000005, rho=0.8171340077735547, optimizer=Nadam, lr=0.0001, layers=4, epsilon=1e-06, epochs=100, decay=0.007, beta_2=0.9446772765480753, beta_1=0.9288119327973722, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8>, score=0.034688, total=  11.4s
[CV] units4=32, units3=64, units2=32, units1=4, schedule_decay=0.008, rho=0.8753061375171187, optimizer=SGD, lr=0.0001, layers=1, epsilon=1e-09, epochs=100, decay=0.059000000000000004, beta_2=0.9643806747145691, beta_1=0.9423346068550754, activation=relu 
[CV]  units4=32, units3=64, units2=32, units1=4, schedule_decay=0.008, rho=0.8753061375171187, optimizer=SGD, lr=0.0001, layers=1, epsilon=1e-09, epochs=100, decay=0.059000000000000004, beta_2=0.9643806747145691, beta_1=0.9423346068550754, activation=relu, score=0.057604, total=  10.0s
[CV] units4=32, units3=64, units2=32, units1=4, schedule_decay=0.008, rho=0.8753061375171187, optimizer=SGD, lr

[CV]  units4=16, units3=4, units2=8, units1=4, schedule_decay=0.0055000000000000005, rho=0.9471585230297452, optimizer=SGD, lr=0.0001, layers=4, epsilon=1e-08, epochs=100, decay=0.002, beta_2=0.926171906571784, beta_1=0.9432879733220609, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8>, score=0.039449, total=  11.3s
[CV] units4=16, units3=4, units2=8, units1=4, schedule_decay=0.0055000000000000005, rho=0.9471585230297452, optimizer=SGD, lr=0.0001, layers=4, epsilon=1e-08, epochs=100, decay=0.002, beta_2=0.926171906571784, beta_1=0.9432879733220609, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8> 
[CV]  units4=16, units3=4, units2=8, units1=4, schedule_decay=0.0055000000000000005, rho=0.9471585230297452, optimizer=SGD, lr=0.0001, layers=4, epsilon=1e-08, epochs=100, decay=0.002, beta_2=0.926171906571784, beta_1=0.9432879733220609, activation=<keras.layers.advanced_activations.ELU object at 0x13259c0b8>, score=0.039946, total=  11.6s
[CV] 

[CV]  units4=16, units3=8, units2=32, units1=32, schedule_decay=0.004, rho=0.8576629684993036, optimizer=Adadelta, lr=0.01, layers=3, epsilon=0.0001, epochs=100, decay=0.001, beta_2=0.936669600294371, beta_1=0.9340763715814911, activation=relu, score=0.037166, total=  12.6s
[CV] units4=16, units3=8, units2=32, units1=32, schedule_decay=0.004, rho=0.8576629684993036, optimizer=Adadelta, lr=0.01, layers=3, epsilon=0.0001, epochs=100, decay=0.001, beta_2=0.936669600294371, beta_1=0.9340763715814911, activation=relu 
[CV]  units4=16, units3=8, units2=32, units1=32, schedule_decay=0.004, rho=0.8576629684993036, optimizer=Adadelta, lr=0.01, layers=3, epsilon=0.0001, epochs=100, decay=0.001, beta_2=0.936669600294371, beta_1=0.9340763715814911, activation=relu, score=0.040628, total=  12.8s
[CV] units4=32, units3=4, units2=4, units1=32, schedule_decay=0.0045000000000000005, rho=0.8070527841734456, optimizer=Adagrad, lr=0.01, layers=1, epsilon=0.0001, epochs=100, decay=0.092, beta_2=0.904467101

In [17]:
results = sweep_stats(sweeper)
# FILE_PATH = os.path.join(CWD, "../results/best_params/forecasting/tCNN_hourlyData-25percent_SEEDFEATS-DP-RH-WindGust-Speed_Resp-ForeCast-LW-8-2-VWC_H1_Rw120_W360_Tune_run1.csv")
# results.to_csv(FILE_PATH)
results

,Score,Std,activation,beta_1,beta_2,decay,epochs,epsilon,layers,lr,optimizer,rho,schedule_decay,units1,units2,units3,units4
0,0.059399,0.012973,relu,0.942335,0.964381,0.059,100,1.000000e-09,1,0.00010,SGD,0.875306,0.0080,4,32,64,32
1,0.053150,0.010719,relu,0.942335,0.950682,0.051,100,1.000000e-08,1,0.00001,Adam,0.928422,0.0065,8,64,4,32
2,0.052239,0.003560,relu,0.927009,0.976113,0.068,100,1.000000e-08,1,0.00001,Adam,0.831176,0.0040,26,16,64,8
3,0.052008,0.008344,<keras.layers.advanced_activations.ELU object ...,0.934076,0.904467,0.080,100,1.000000e-05,1,0.00001,Adagrad,0.907762,0.0070,4,32,32,4
4,0.049727,0.003295,<keras.layers.advanced_activations.ELU object ...,0.927009,0.950682,0.015,100,1.000000e-05,4,0.00001,Adamax,0.916471,0.0090,4,4,4,4
5,0.049717,0.003479,<keras.layers.advanced_activations.ELU object ...,0.936877,0.976113,0.062,100,1.000000e-07,2,0.00010,SGD,0.943074,0.0035,4,16,32,64
6,0.049205,0.002626,<keras.layers.advanced_activations.ELU object ...,0.942335,0.944677,0.066,100,1.000000e-04,1,0.00001,RMSprop,0.831176,0.0090,26,64,16,8
7,0.048971,0.008862,<keras.layers.advanced_activations.ELU object ...,0.987881,0.983522,0.033,100,1.000000e-08,3,0.00001,Adam,0.947159,0.0050,32,32,32,8
8,0.048960,0.004574,<keras.layers.advanced_activations.ELU object ...,0.928812,0.910159,0.035,100,1.000000e-09,2,0.00100,Adadelta,0.850157,0.0030,4,4,64,16
9,0.048956,0.004099,<keras.layers.advanced_activations.ELU object ...,0.930683,0.950682,0.089,100,1.000000e-06,1,0.00100,Adam,0.866608,0.0080,8,64,16,16


In [21]:
for i in range(10):

    RUN = i+1
    print('Best Run Num: {0}'.format(RUN))
    params = results.iloc[-RUN,2:].to_dict()
    # params

    ### Testing the best params

    np.random.seed(84)
    tf.set_random_seed(84)

    params['epochs'] = 100

    train_results, test_results = [], []
    ITERATIONS=3
    VERBOSE=0

    # params

#     early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=25, min_lr=1e-08, 
                                  mode='auto', verbose=1)

    for i in range(ITERATIONS):
        model = KerasRegressor(build_fn=create_NN_model, shuffle=True, verbose=VERBOSE, **params)
        history = model.fit(X_train, y_train, verbose=VERBOSE, validation_split = 0.15, callbacks=[reduce_lr])
#         history = model.fit(X_train, y_train, verbose=VERBOSE, validation_split = None)
        train_results.append(test_mae(model)[0])
        test_results.append(test_mae(model)[1])
    report(train_results, test_results)

Best Run Num: 1

Epoch 00074: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00099: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00062: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00087: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00045: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00070: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00095: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.


('Training MAE 0.023253 ± 0.000450', 'Testing MAE 0.032841 ± 0.001472')

Best Run Num: 2

Epoch 00073: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00098: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00034: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00059: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00084: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.


('Training MAE 0.029665 ± 0.000799', 'Testing MAE 0.032440 ± 0.001228')

Best Run Num: 3


...

('Training MAE 0.027999 ± 0.000978', 'Testing MAE 0.034873 ± 0.003267')

Best Run Num: 9

Epoch 00040: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00065: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00090: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.

Epoch 00067: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00092: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00090: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.


('Training MAE 0.027544 ± 0.000484', 'Testing MAE 0.036058 ± 0.001192')

Best Run Num: 10


('Training MAE 0.038170 ± 0.000080', 'Testing MAE 0.042772 ± 0.001089')

In [22]:
RUN = 1
print('Best Run Num: {0}'.format(RUN))
params = results.iloc[-RUN,2:].to_dict()

### Testing the best params

np.random.seed(84)
tf.set_random_seed(84)

params['epochs'] = 100

train_results, test_results = [], []
ITERATIONS=20
VERBOSE=0

#     early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=25, min_lr=1e-08, 
                              mode='auto', verbose=0)

for i in range(ITERATIONS):
    print(i)
    model = KerasRegressor(build_fn=create_NN_model, shuffle=True, verbose=VERBOSE, **params)
    history = model.fit(X_train, y_train, verbose=VERBOSE, validation_split = 0.15, callbacks=[reduce_lr])
#         history = model.fit(X_train, y_train, verbose=VERBOSE, validation_split = None)
    train_results.append(test_mae(model)[0])
    test_results.append(test_mae(model)[1])
report(train_results, test_results)

Best Run Num: 1
0

Epoch 00074: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00099: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
1

Epoch 00059: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00096: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
2

Epoch 00045: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00070: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.

Epoch 00095: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.
3

Epoch 00059: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00084: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
4

Epoch 00082: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
5

Epoch 00035: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

Epoch 00060: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.


('Training MAE 0.023763 ± 0.001320', 'Testing MAE 0.032601 ± 0.001573')

In [25]:
mae_df_NN = pd.DataFrame({'mae_NN': test_results})
mae_df_NN.to_csv('../data/results_data/results_df_NN.csv')

In [ ]:
RUN = 1

params = results.iloc[-RUN,2:].to_dict()

### Testing the best params

np.random.seed(84)
tf.set_random_seed(84)

params['epochs'] = 100

train_results, test_results = [], []
ITERATIONS=3
VERBOSE=0

# params

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-08, mode='auto', verbose=1)

for i in range(ITERATIONS):
    model = KerasRegressor(build_fn=create_NN_model, shuffle=True, verbose=VERBOSE, **params)
#     history = model.fit(train_X, train_y, verbose=VERBOSE, validation_split = 0.15, callbacks=[early_stopping])
    history = model.fit(X_train, y_train, verbose=VERBOSE, validation_split = None)
    train_results.append(test_mae(model)[0])
    test_results.append(test_mae(model)[1])
    report(train_results, test_results)